In [1]:
import yaml
import numpy as np
import matplotlib
import csv
import sys
import pandas as pd

from tkinter import filedialog
from PyQt5 import QtCore, QtWidgets
from matplotlib.figure import Figure
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar

from PyQt5.QtWidgets import QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton, QFileDialog

matplotlib.use('Qt5Agg')

In [2]:
def Gaussian_Dist(mean, stddev):
    return np.random.normal(loc=mean, scale=stddev)

def Beta_Dist(a, b):
    return np.random.beta(a, b)

def Lognormal_Dist(log_mean, sigma):
    return np.random.lognormal(mean=log_mean, sigma=sigma)

In [3]:
def computed_LP(mileage, SOCe, t0, Tf, Pm, E0, Pch0, eta, mode):
    LP = np.zeros(2880)

    Em = mileage * Pm
    Em_m = Em*60
    E0_m = E0*60000
    Tf_m = Tf*60

    Pch = Pch0*eta/100
    Pch_m = Pch*1000

    t0_m = t0*60

    if (mode == 1):                 # final SOC
        Ech = Em_m
        Tch = Ech/Pch_m
        np.ceil(Tch).astype(int)
        T = t0_m + Tch
        Tch = np.ceil(Tch)
        T = np.ceil(T)

    else:
        SOCi = (SOCe/100 - Em_m/E0_m)
        Ech = (1 - SOCi)*E0_m
        Tch = Ech/Pch_m
        if (Tf_m > Tch):
            T = t0_m + Tch
        else:
            T = t0_m + Tf
        Tch = np.ceil(Tch)
        T = np.ceil(T)

    LP[t0_m: int(np.ceil(T))] = Pch_m

    return LP, Tch, T

In [4]:
def get_customer_probability(customers_data, customer):
    if(customer == 'All' or customer == ''):
        customers_propability = np.array([])
        for cus in list(customers_data.keys())[1:]:
            customers_propability = np.append(customers_propability, customers_data[cus]['probability'])
        
    else:
        customers_propability = np.zeros(len(list(customers_data.keys())[1:]))
        idx = list(customers_data.keys())[1:].index(customer)
        customers_propability[idx] = 1
    
    return customers_propability

In [5]:
def get_type_prob(customers_data, customer):
    type_propability = np.array([])
    for type in list(customers_data[customer].keys())[:-1]:
        type_propability = np.append(type_propability, customers_data[customer][type]['probability'])
    
    return type_propability

In [6]:
def write_csv_process(all_distributions, customers_data, customer):
    Vehicle_database = pd.read_csv('data.csv')

    all_custormers = list(customers_data.keys())[1:]
    customers_propability = get_customer_probability(customers_data, customer)
    
    N = customers_data['Number_of_sample']

    # load profile initialization
    Load_profile = []
    Total_LP = np.zeros(2880)

    # open the csv file for recording data of individual vehicle
    with open('Individual_charging_data.csv', 'w', newline='\n') as cfile:
        writer = csv.writer(cfile, delimiter=',')
        elements = [" Customer Type ", " Electric Vehicle Type ", " mileage ", " SOC ", " Start Time of Charging "," Fixed Charging duration ", " Calculated Charging duration ", " End Time of Charging "]
        writer.writerow(elements)

        for i in range(N):
            # Sample from customers
            cust = np.random.choice(all_custormers, 1, p=customers_propability)
            cust_data = customers_data[cust[0]]

            # discrete disttribution of customer's vehicle
            V = list(cust_data.keys())[0:-1]

            # probability = cust_data['Probability']
            probability = get_type_prob(customers_data, cust[0])

            Vehicle = np.random.choice(V, 1, p=probability)
            cust_vehicle_data = cust_data[Vehicle[0]]
            
            # read the distribution
            dist_mileage = cust_vehicle_data['mileage']
            dist_SOC     = cust_vehicle_data['SOC']
            dist_t0      = cust_vehicle_data['t0']
            dist_Tf      = cust_vehicle_data['Tf']
            mode         = cust_vehicle_data['mode']

            # sample from distribution        
            mileage = int(all_distributions[dist_mileage['Distribution']](**dist_mileage['Parameters']))
            SOC     = int(all_distributions[dist_SOC['Distribution']](**dist_SOC['Parameters']))
            t0      = int(all_distributions[dist_t0['Distribution']](**dist_t0['Parameters']))
            Tf      = int(all_distributions[dist_Tf['Distribution']](**dist_Tf['Parameters']))

            # vehicle parameters
            Pm = Vehicle_database.loc[Vehicle_database["Name"] == Vehicle[0], "Power consumption (Wh/km)"].values[0]
            E0 = Vehicle_database.loc[Vehicle_database["Name"] == Vehicle[0], "Battery capacity(kWh)"].values[0]
            Pch0 = Vehicle_database.loc[Vehicle_database["Name"] == Vehicle[0], "Charging power (kW)"].values[0]
            eta = 100

            # load profile calculation (single vehicle)
            Lp, Tch, T = computed_LP(mileage, SOC, t0, Tf, Pm, E0, Pch0, eta, mode)
            Load_profile.append(Lp)
            csv_write_distribution = [cust[0], Vehicle[0], mileage, SOC, t0, Tf, Tch, T]

            # write into csv:
            writer.writerow(csv_write_distribution)

            # Total load profile accumulation
            Total_LP = np.add(Total_LP, Load_profile[i])
    cfile.close()

    return Total_LP

In [7]:
# Read master customer data file YAML
def read_data(file_name):
    with open(file_name) as file:
        customers_data = yaml.safe_load(file)
    return customers_data

# some initialization
all_distributions = {'Gaussian': Gaussian_Dist,
                     'Beta': Beta_Dist, 
                     'Lognormal': Lognormal_Dist}

all_parameters = {
    'Gaussian':  ['Mean', 'Standard deviation'],
    'Beta':      ['a', 'b'],
    'Lognormal': ['Mean', 'Standard deviation']
}

In [8]:
class Ui_MainWindow(object):

    def setupUi(self, MainWindow, customers_data):
        self.customers_data = customers_data

        # INTERFACE
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(1057, 606)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(260, 10, 791, 551))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.verticalLayout = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.verticalLayout.setContentsMargins(0, 0, 0, 0)
        self.verticalLayout.setObjectName("verticalLayout")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(10, 10, 101, 21))
        self.label.setObjectName("label")
        self.lineEdit_number_of_sample = QtWidgets.QLineEdit(self.centralwidget)
        self.lineEdit_number_of_sample.setGeometry(QtCore.QRect(122, 10, 121, 21))
        self.lineEdit_number_of_sample.setObjectName("lineEdit_number_of_sample")
        self.groupBox = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox.setGeometry(QtCore.QRect(10, 80, 231, 81))
        self.groupBox.setObjectName("groupBox")
        self.cBox_CustomerType = QtWidgets.QComboBox(self.groupBox)
        self.cBox_CustomerType.setGeometry(QtCore.QRect(130, 40, 91, 21))
        self.cBox_CustomerType.setObjectName("cBox_CustomerType")
        self.label_2 = QtWidgets.QLabel(self.groupBox)
        self.label_2.setGeometry(QtCore.QRect(20, 40, 111, 21))
        self.label_2.setObjectName("label_2")
        self.label_3 = QtWidgets.QLabel(self.groupBox)
        self.label_3.setGeometry(QtCore.QRect(10, 20, 51, 21))
        self.label_3.setObjectName("label_3")
        self.cBox_Customer = QtWidgets.QComboBox(self.groupBox)
        self.cBox_Customer.setGeometry(QtCore.QRect(90, 20, 131, 21))
        self.cBox_Customer.setObjectName("cBox_Customer")
        self.cBox_CustomerMode = QtWidgets.QComboBox(self.groupBox)
        self.cBox_CustomerMode.setGeometry(QtCore.QRect(130, 60, 91, 21))
        self.cBox_CustomerMode.setObjectName("cBox_CustomerMode")
        self.label_4 = QtWidgets.QLabel(self.groupBox)
        self.label_4.setGeometry(QtCore.QRect(20, 60, 111, 21))
        self.label_4.setObjectName("label_4")
        self.groupBox_2 = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox_2.setGeometry(QtCore.QRect(10, 170, 231, 81))
        self.groupBox_2.setObjectName("groupBox_2")
        self.label_Mileage_para1 = QtWidgets.QLabel(self.groupBox_2)
        self.label_Mileage_para1.setGeometry(QtCore.QRect(20, 40, 101, 20))
        self.label_Mileage_para1.setObjectName("label_Mileage_para1")
        self.label_8 = QtWidgets.QLabel(self.groupBox_2)
        self.label_8.setGeometry(QtCore.QRect(10, 19, 71, 21))
        self.label_8.setObjectName("label_8")
        self.lineEdit_Mileage_para1 = QtWidgets.QLineEdit(self.groupBox_2)
        self.lineEdit_Mileage_para1.setGeometry(QtCore.QRect(130, 40, 91, 20))
        self.lineEdit_Mileage_para1.setObjectName("lineEdit_Mileage_para1")
        self.lineEdit_Mileage_para2 = QtWidgets.QLineEdit(self.groupBox_2)
        self.lineEdit_Mileage_para2.setGeometry(QtCore.QRect(130, 59, 91, 21))
        self.lineEdit_Mileage_para2.setObjectName("lineEdit_Mileage_para2")
        self.label_Mileage_para2 = QtWidgets.QLabel(self.groupBox_2)
        self.label_Mileage_para2.setGeometry(QtCore.QRect(20, 60, 101, 20))
        self.label_Mileage_para2.setObjectName("label_Mileage_para2")
        self.cBox_MileageDist = QtWidgets.QComboBox(self.groupBox_2)
        self.cBox_MileageDist.setGeometry(QtCore.QRect(90, 19, 131, 21))
        self.cBox_MileageDist.setObjectName("cBox_MileageDist")
        self.groupBox_3 = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox_3.setGeometry(QtCore.QRect(10, 260, 231, 81))
        self.groupBox_3.setObjectName("groupBox_3")
        self.label_SOC_para1 = QtWidgets.QLabel(self.groupBox_3)
        self.label_SOC_para1.setGeometry(QtCore.QRect(20, 40, 101, 20))
        self.label_SOC_para1.setObjectName("label_SOC_para1")
        self.label_12 = QtWidgets.QLabel(self.groupBox_3)
        self.label_12.setGeometry(QtCore.QRect(10, 19, 71, 21))
        self.label_12.setObjectName("label_12")
        self.lineEdit_SOC_para1 = QtWidgets.QLineEdit(self.groupBox_3)
        self.lineEdit_SOC_para1.setGeometry(QtCore.QRect(130, 40, 91, 20))
        self.lineEdit_SOC_para1.setObjectName("lineEdit_SOC_para1")
        self.lineEdit_SOC_para2 = QtWidgets.QLineEdit(self.groupBox_3)
        self.lineEdit_SOC_para2.setGeometry(QtCore.QRect(130, 59, 91, 21))
        self.lineEdit_SOC_para2.setObjectName("lineEdit_SOC_para2")
        self.label_SOC_para2 = QtWidgets.QLabel(self.groupBox_3)
        self.label_SOC_para2.setGeometry(QtCore.QRect(20, 60, 101, 20))
        self.label_SOC_para2.setObjectName("label_SOC_para2")
        self.cBox_SOCDist = QtWidgets.QComboBox(self.groupBox_3)
        self.cBox_SOCDist.setGeometry(QtCore.QRect(90, 19, 131, 21))
        self.cBox_SOCDist.setObjectName("cBox_SOCDist")
        self.groupBox_4 = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox_4.setGeometry(QtCore.QRect(10, 350, 231, 81))
        self.groupBox_4.setObjectName("groupBox_4")
        self.label_t0_para1 = QtWidgets.QLabel(self.groupBox_4)
        self.label_t0_para1.setGeometry(QtCore.QRect(20, 40, 101, 20))
        self.label_t0_para1.setObjectName("label_t0_para1")
        self.label_15 = QtWidgets.QLabel(self.groupBox_4)
        self.label_15.setGeometry(QtCore.QRect(10, 19, 71, 21))
        self.label_15.setObjectName("label_15")
        self.lineEdit_t0_para1 = QtWidgets.QLineEdit(self.groupBox_4)
        self.lineEdit_t0_para1.setGeometry(QtCore.QRect(130, 40, 91, 20))
        self.lineEdit_t0_para1.setObjectName("lineEdit_t0_para1")
        self.lineEdit_t0_para2 = QtWidgets.QLineEdit(self.groupBox_4)
        self.lineEdit_t0_para2.setGeometry(QtCore.QRect(130, 59, 91, 21))
        self.lineEdit_t0_para2.setObjectName("lineEdit_t0_para2")
        self.label_t0_para2 = QtWidgets.QLabel(self.groupBox_4)
        self.label_t0_para2.setGeometry(QtCore.QRect(20, 60, 101, 20))
        self.label_t0_para2.setObjectName("label_t0_para2")
        self.cBox_t0Dist = QtWidgets.QComboBox(self.groupBox_4)
        self.cBox_t0Dist.setGeometry(QtCore.QRect(90, 19, 131, 21))
        self.cBox_t0Dist.setObjectName("cBox_t0Dist")
        self.groupBox_5 = QtWidgets.QGroupBox(self.centralwidget)
        self.groupBox_5.setGeometry(QtCore.QRect(10, 440, 231, 81))
        self.groupBox_5.setObjectName("groupBox_5")
        self.label_Tf_para1 = QtWidgets.QLabel(self.groupBox_5)
        self.label_Tf_para1.setGeometry(QtCore.QRect(20, 40, 101, 20))
        self.label_Tf_para1.setObjectName("label_Tf_para1")
        self.label_18 = QtWidgets.QLabel(self.groupBox_5)
        self.label_18.setGeometry(QtCore.QRect(10, 19, 71, 21))
        self.label_18.setObjectName("label_18")
        self.lineEdit_Tf_para1 = QtWidgets.QLineEdit(self.groupBox_5)
        self.lineEdit_Tf_para1.setGeometry(QtCore.QRect(130, 40, 91, 20))
        self.lineEdit_Tf_para1.setObjectName("lineEdit_Tf_para1")
        self.lineEdit_Tf_para2 = QtWidgets.QLineEdit(self.groupBox_5)
        self.lineEdit_Tf_para2.setGeometry(QtCore.QRect(130, 59, 91, 21))
        self.lineEdit_Tf_para2.setObjectName("lineEdit_Tf_para2")
        self.label_Tf_para2 = QtWidgets.QLabel(self.groupBox_5)
        self.label_Tf_para2.setGeometry(QtCore.QRect(20, 60, 101, 20))
        self.label_Tf_para2.setObjectName("label_Tf_para2")
        self.cBox_TfDist = QtWidgets.QComboBox(self.groupBox_5)
        self.cBox_TfDist.setGeometry(QtCore.QRect(90, 19, 131, 21))
        self.cBox_TfDist.setObjectName("cBox_TfDist")
        self.Btn_Simulate = QtWidgets.QPushButton(self.centralwidget)
        self.Btn_Simulate.setGeometry(QtCore.QRect(150, 540, 61, 21))
        self.Btn_Simulate.setObjectName("Btn_Simulate")
        self.Btn_Export = QtWidgets.QPushButton(self.centralwidget)
        self.Btn_Export.setGeometry(QtCore.QRect(40, 540, 61, 21))
        self.Btn_Export.setObjectName("Btn_Export")
        self.label_5 = QtWidgets.QLabel(self.centralwidget)
        self.label_5.setGeometry(QtCore.QRect(10, 40, 101, 21))
        self.label_5.setObjectName("label_5")
        self.cBox_Customer_2 = QtWidgets.QComboBox(self.centralwidget)
        self.cBox_Customer_2.setGeometry(QtCore.QRect(100, 40, 141, 21))
        self.cBox_Customer_2.setObjectName("cBox_Customer_2")
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 1057, 22))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

        # --------------------------------------------------
        self.lineEdit_number_of_sample.setText(str(self.customers_data['Number_of_sample']))

        self.cBox_Customer_2.addItem("All")
        self.cBox_Customer_2.addItems(list(self.customers_data.keys())[1:])
        self.cBox_Customer.addItems(list(self.customers_data.keys())[1:])
        self.cBox_Customer.currentIndexChanged.connect(self.cBox_Customer_change)
        self.cBox_CustomerType.addItems(list(self.customers_data[self.cBox_Customer.currentText()].keys())[0:-1])
        self.cBox_CustomerType.currentIndexChanged.connect(self.cBox_CustomerType_change)
        self.cBox_CustomerMode.addItems(["1", '0'])
        self.cBox_CustomerMode.setCurrentText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['mode']))

        # Group Mileage
        self.cBox_MileageDist.addItems(list(all_distributions.keys()))
        self.cBox_MileageDist.setCurrentText(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['mileage']['Distribution'])
        self.cBox_MileageDist.currentIndexChanged.connect(self.cBox_MileageDist_change)
        mileage_para1 = list(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['mileage']['Parameters'].keys())[0]
        mileage_para2 = list(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['mileage']['Parameters'].keys())[1]
        self.label_Mileage_para1.setText(mileage_para1)
        self.label_Mileage_para2.setText(mileage_para2)
        self.lineEdit_Mileage_para1.setText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['mileage']['Parameters'][mileage_para1]))
        self.lineEdit_Mileage_para2.setText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['mileage']['Parameters'][mileage_para2]))

        # Group SOC
        self.cBox_SOCDist.addItems(list(all_distributions.keys()))
        self.cBox_SOCDist.setCurrentText(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['SOC']['Distribution'])
        self.cBox_SOCDist.currentIndexChanged.connect(self.cBox_SOCDist_change)
        SOC_para1 = list(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['SOC']['Parameters'].keys())[0]
        SOC_para2 = list(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['SOC']['Parameters'].keys())[1]
        self.lineEdit_SOC_para1.setText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['SOC']['Parameters'][SOC_para1]))
        self.lineEdit_SOC_para2.setText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['SOC']['Parameters'][SOC_para2]))
        
        # Group t0
        self.cBox_t0Dist.addItems(list(all_distributions.keys()))
        self.cBox_t0Dist.setCurrentText(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['t0']['Distribution'])
        self.cBox_t0Dist.currentIndexChanged.connect(self.cBox_t0Dist_change)
        t0_para1 = list(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['t0']['Parameters'].keys())[0]
        t0_para2 = list(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['t0']['Parameters'].keys())[1]
        self.lineEdit_t0_para1.setText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['t0']['Parameters'][t0_para1]))
        self.lineEdit_t0_para2.setText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['t0']['Parameters'][t0_para2]))
        
        # Group Tf
        self.cBox_TfDist.addItems(list(all_distributions.keys()))
        self.cBox_TfDist.setCurrentText(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['Tf']['Distribution'])
        self.cBox_TfDist.currentIndexChanged.connect(self.cBox_TfDist_change)
        Tf_para1 = list(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['Tf']['Parameters'].keys())[0]
        Tf_para2 = list(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['Tf']['Parameters'].keys())[1]
        self.lineEdit_Tf_para1.setText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['Tf']['Parameters'][Tf_para1]))
        self.lineEdit_Tf_para2.setText(str(self.customers_data[self.cBox_Customer.currentText()][self.cBox_CustomerType.currentText()]['Tf']['Parameters'][Tf_para2]))

        # button
        self.Btn_Export.clicked.connect(self.export)
        self.Btn_Simulate.clicked.connect(self.simulate)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.label.setText(_translate("MainWindow", "Number of sample:"))
        self.groupBox.setTitle(_translate("MainWindow", "Customer Data"))
        self.label_2.setText(_translate("MainWindow", "Type:"))
        self.label_3.setText(_translate("MainWindow", "Customer:"))
        self.label_4.setText(_translate("MainWindow", "Mode:"))
        self.groupBox_2.setTitle(_translate("MainWindow", "Mileage"))
        self.label_Mileage_para1.setText(_translate("MainWindow", "Mean"))
        self.label_8.setText(_translate("MainWindow", "Distribution"))
        self.label_Mileage_para2.setText(_translate("MainWindow", "Standard deviation"))
        self.groupBox_3.setTitle(_translate("MainWindow", "SOC"))
        self.label_SOC_para1.setText(_translate("MainWindow", "Mean"))
        self.label_12.setText(_translate("MainWindow", "Distribution"))
        self.label_SOC_para2.setText(_translate("MainWindow", "Standard deviation"))
        self.groupBox_4.setTitle(_translate("MainWindow", "Initial time"))
        self.label_t0_para1.setText(_translate("MainWindow", "Mean"))
        self.label_15.setText(_translate("MainWindow", "Distribution"))
        self.label_t0_para2.setText(_translate("MainWindow", "Standard deviation"))
        self.groupBox_5.setTitle(_translate("MainWindow", "Charging time"))
        self.label_Tf_para1.setText(_translate("MainWindow", "Mean"))
        self.label_18.setText(_translate("MainWindow", "Distribution"))
        self.label_Tf_para2.setText(_translate("MainWindow", "Standard deviation"))
        self.Btn_Simulate.setText(_translate("MainWindow", "Simulate"))
        self.Btn_Export.setText(_translate("MainWindow", "Export"))
        self.label_5.setText(_translate("MainWindow", "Customer:"))
    
    
    def cBox_Customer_change(self):
        self.cBox_CustomerType.clear()
        self.cBox_CustomerType.addItems(list(self.customers_data[self.cBox_Customer.currentText()].keys())[0:-1])
        
        
    def cBox_CustomerType_change(self):
        cus_type = self.cBox_CustomerType.currentText()
        if(cus_type == ''):
            cus_type = list(self.customers_data[self.cBox_Customer.currentText()].keys())[0]
        
        self.cBox_CustomerMode.setCurrentText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['mode']))

        self.cBox_MileageDist.setCurrentText(self.customers_data[self.cBox_Customer.currentText()][cus_type]['mileage']['Distribution'])
        mileage_para1 = list(self.customers_data[self.cBox_Customer.currentText()][cus_type]['mileage']['Parameters'].keys())[0]
        mileage_para2 = list(self.customers_data[self.cBox_Customer.currentText()][cus_type]['mileage']['Parameters'].keys())[1]
        self.lineEdit_Mileage_para1.setText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['mileage']['Parameters'][mileage_para1]))
        self.lineEdit_Mileage_para2.setText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['mileage']['Parameters'][mileage_para2]))

        self.cBox_SOCDist.setCurrentText(self.customers_data[self.cBox_Customer.currentText()][cus_type]['SOC']['Distribution'])
        SOC_para1 = list(self.customers_data[self.cBox_Customer.currentText()][cus_type]['SOC']['Parameters'].keys())[0]
        SOC_para2 = list(self.customers_data[self.cBox_Customer.currentText()][cus_type]['SOC']['Parameters'].keys())[1]
        self.lineEdit_SOC_para1.setText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['SOC']['Parameters'][SOC_para1]))
        self.lineEdit_SOC_para2.setText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['SOC']['Parameters'][SOC_para2]))

        self.cBox_t0Dist.setCurrentText(self.customers_data[self.cBox_Customer.currentText()][cus_type]['t0']['Distribution'])
        t0_para1 = list(self.customers_data[self.cBox_Customer.currentText()][cus_type]['t0']['Parameters'].keys())[0]
        t0_para2 = list(self.customers_data[self.cBox_Customer.currentText()][cus_type]['t0']['Parameters'].keys())[1]
        self.lineEdit_t0_para1.setText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['t0']['Parameters'][t0_para1]))
        self.lineEdit_t0_para2.setText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['t0']['Parameters'][t0_para2]))

        self.cBox_TfDist.setCurrentText(self.customers_data[self.cBox_Customer.currentText()][cus_type]['Tf']['Distribution'])
        Tf_para1 = list(self.customers_data[self.cBox_Customer.currentText()][cus_type]['Tf']['Parameters'].keys())[0]
        Tf_para2 = list(self.customers_data[self.cBox_Customer.currentText()][cus_type]['Tf']['Parameters'].keys())[1]
        self.lineEdit_Tf_para1.setText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['Tf']['Parameters'][Tf_para1]))
        self.lineEdit_Tf_para2.setText(str(self.customers_data[self.cBox_Customer.currentText()][cus_type]['Tf']['Parameters'][Tf_para2]))


    def cBox_MileageDist_change(self):
        dist = self.cBox_MileageDist.currentText()
        mileage_para1 = all_parameters[dist][0]
        mileage_para2 = all_parameters[dist][1]
        self.label_Mileage_para1.setText(mileage_para1)
        self.label_Mileage_para2.setText(mileage_para2)
    
    def cBox_SOCDist_change(self):
        dist = self.cBox_SOCDist.currentText()
        mileage_para1 = all_parameters[dist][0]
        mileage_para2 = all_parameters[dist][1]
        self.label_SOC_para1.setText(mileage_para1)
        self.label_SOC_para2.setText(mileage_para2)
    
    def cBox_t0Dist_change(self):
        dist = self.cBox_t0Dist.currentText()
        mileage_para1 = all_parameters[dist][0]
        mileage_para2 = all_parameters[dist][1]
        self.label_t0_para1.setText(mileage_para1)
        self.label_t0_para2.setText(mileage_para2)
    
    def cBox_TfDist_change(self):
        dist = self.cBox_TfDist.currentText()
        mileage_para1 = all_parameters[dist][0]
        mileage_para2 = all_parameters[dist][1]
        self.label_Tf_para1.setText(mileage_para1)
        self.label_Tf_para2.setText(mileage_para2)


    def export(self):
        filename, _ = QFileDialog.getSaveFileName(filter="CSV Files (*.csv)")
        if filename:
            with open(filename, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(self.Total_LP)
        
    
    def simulate(self):
        # clear
        for i in reversed(range(self.verticalLayout.count())): 
            self.verticalLayout.itemAt(i).widget().setParent(None)

        # compute Total_LP and export csv
        # customer = self.cBox_Customer_2.currentIndex()
        customer = self.cBox_Customer_2.currentText()
        self.Total_LP = write_csv_process(all_distributions, self.customers_data, customer)

        # plot
        sc = MplCanvas(self, width=5, height=4, dpi=90)
        x_value = np.arange(0, 2880)
        sc.axes.plot(x_value, self.Total_LP)

        # Create toolbar, passing canvas as first parament, parent (self, the MainWindow) as second.
        toolbar = NavigationToolbar(sc)
        self.verticalLayout.addWidget(toolbar)
        self.verticalLayout.addWidget(sc)


class MplCanvas(FigureCanvasQTAgg):
    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        self.axes.set_ylabel('Load Profile')
        self.axes.set_xlabel('Time')
        super(MplCanvas, self).__init__(fig)

In [9]:
def choose_input_file():
    app = QApplication([])
    filename, _ = QFileDialog.getOpenFileName(None, "Select input file...", "", "YAML Files (*.yml)")
    return filename

In [10]:
if __name__ == "__main__":
    file_path = choose_input_file()
    customers_data = read_data(file_path)
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow, customers_data)
    MainWindow.setWindowTitle("Load Profile Application")
    MainWindow.setFixedSize(1057, 606)
    MainWindow.setStyleSheet("QWidget {font-size: 6.5pt; }")

    MainWindow.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\tln229\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
